# Perkenalan

**Anggota Kelompok**

1. Fachriansyah Muhammad Haikal (PYTN-KS14-010)
2. Fakhrurrozi (PYTN-KS14-018)
3. Safira Alya Fafaza (PYTN-KS14-017)

**Latar Belakang**
Penyakit gagal jantung adalah kondisi yang serius dan mempengaruhi kualitas hidup serta tingkat kematian pasien. Prediksi yang akurat tentang risiko gagal jantung dapat membantu tenaga medis dalam mengidentifikasi pasien yang rentan dan memungkinkan pengambilan tindakan pencegahan yang tepat.

Dalam hal ini, penggunaan ensemble model menjadi relevan karena dapat memanfaatkan kekuatan berbagai algoritma pembelajaran mesin yang berbeda. Ensemble model menggabungkan hasil prediksi dari beberapa model yang berbeda, seperti KNN, random forest, naive bayes, dan lainnya. Dengan memadukan prediksi dari beberapa model, ensemble model dapat mengurangi bias dan varians yang mungkin ada dalam model tunggal, dan pada akhirnya menghasilkan prediksi yang lebih akurat.

Ensemble model memiliki beberapa variasi, seperti voting ensemble, bagging, dan boosting. Misalnya, pada voting ensemble, prediksi akhir didapatkan dengan memilih hasil prediksi mayoritas dari model yang terlibat. Dengan menggabungkan hasil prediksi dari beberapa model yang berbeda, ensemble model pada dataset Heart Failure Prediction dapat meningkatkan akurasi dan performa prediksi. Dalam konteks medis, hal ini akan membantu dalam mengidentifikasi pasien dengan risiko tinggi gagal jantung, sehingga dapat dilakukan tindakan pencegahan yang tepat untuk meningkatkan prognosis dan kualitas hidup mereka.

**Dataset**

Dataset yang digunakan dalam projek ini adalah Heart Failure yang berisi kelangsungan hidup pasien dengan gagal jantung dari kreatinin serum dan fraksi ejeksi. Dataset ini berjumlah 299 dengan 13 atribut. Pada projek ini atribut yang akan digunakan dan diperlukan untuk membuat sistem prediksi sebanyak 13 antara lain sebagai berikut :
1. age : umur pasien
2. anaemia : apakah ada pengurangan haemoglobin
3. creatinine_phosphokinase : level enzim CPK dalam mcg/L
4. diabetes : apakah pasien punya riwayat diabetes
5. ejection_fraction : persentase darah yang meninggalkan jantung dalam persentasi di setiap kontraksi jantung
6. high_blood_pressure : apakah pasien punya darah tinggi
7. platelets : jumlah platelet di darah dalam kiloplatelets/mL
8. serum_creatinine : level serum creatinine di darah dalam mg/dL
9. serum_sodium : level serum sodium di darah dalam mEq/L
10. sex : apakah pasien pria atau wanita
11. smoking : apakah pasien merokok
12. time : waktu dalam hari untuk follow-up
13. DEATH_EVENT : apakah pasien sudah meninggal saat waktu follow-up


**Objektif**
* menemukan faktor-faktor yang mempengaruhi terjadinya hujan di Australia
* menemukan akurasi prediksi keselamtan pasien dari penyakit jantung menggunakan konsep Classification dengan Ensemble Model
* memberikan wawasan yang dapat bermanfaat bagi masyarakat mengenai penyakit jantung

# Import Library

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier, VotingClassifier
from sklearn.metrics import confusion_matrix

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import calendar
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'imblearn'

# Data Loading

In [ ]:
df = pd.read_csv('heart_failure_clinical_records_dataset.csv')
df.head()

In [ ]:
# jumlah data dan kolom
df.shape

Dapat diketahui bahwa terdapat 299 baris dan 13 kolom

In [ ]:
df.info()

# Data Cleansing

## Mengganti Nama Atribut

In [ ]:
df.rename(columns={
    'DEATH_EVENT' : 'death_event'
}, inplace=True)
df.head(10)

## Mengubah Isi Atribut

### Yes dan No

In [ ]:
convert_yes_no = ['anaemia', 'diabetes', 'high_blood_pressure', 'smoking', 'death_event']
value_change = {0: 'No', 1: 'Yes'}
df[convert_yes_no] = df[convert_yes_no].replace(value_change)
df

### Woman dan Man

In [ ]:
convert_sex = ['sex']
value_change = {0: 'Woman', 1: 'Man'}
df[convert_sex] = df[convert_sex].replace(value_change)
df

## Cek Data Kosong

In [ ]:
df.isnull().sum()

Dapat diketahui tidak ada data yang kosong

## Cek Data Duplikat

In [ ]:
df.duplicated().sum()

Dapat diketahui tidak ada data yang duplikat

## Cek One Value Unique

In [ ]:
for column in df.columns:
    print(column, df[column].nunique())

Dapat diketahui bahwa atribut yang ada tidak ditemukan one unique value sehingga tidak perlu ada yang dihapus.

# Eksplorasi Data

In [ ]:
df.info()

## Analisa Desktiptif

In [ ]:
df.describe()

## Categorical Features

In [ ]:
categorical_features = [feature for feature in df.columns if (
    df[feature].dtypes == "O")]
categorical_features

In [ ]:
for feature in categorical_features:
    print(
        f"Feature {feature} memiliki unique value {(df[feature].nunique())}")

In [ ]:
# visualisasi categorical features
plt.figure(figsize=(15, 100), facecolor="white")
plotnumb = 1
for cat in categorical_features:
    ax = plt.subplot(15, 2, plotnumb)
    sns.countplot(y=cat, data=df)
    ax.bar_label(ax.containers[0], fontsize=10)
    plt.xlabel(cat)
    plt.title(cat)
    plotnumb += 1
plt.show()

Dapat diketahui bahwa :
1. Orang yang memiliki penyakit jantung kebanyakan berjenis kelamin laki-laki
2. Orang yang memiliki penyakit jantung kebanyakan tidak meninggal

## Numerical Features

In [ ]:
numerical_features = [feature for feature in df.columns if (
    df[feature].dtypes != "O")]
print(numerical_features)
print(f"Terdapat {len(numerical_features)} Numerical Features")

In [ ]:
discrete_feature = [
    feature for feature in numerical_features if df[feature].nunique() < 25]
print(discrete_feature)
print(f"Terdapat {len(discrete_feature)} Discrete Numerical Features")

In [ ]:
continuous_features = [
    feature for feature in numerical_features if feature not in discrete_feature]
print(continuous_features)
print(f"Terdapat {len(continuous_features)} Continuous Feature")

In [ ]:
# visualisasi numerical features
plt.figure(figsize=(20, 60), facecolor='white')
num = 1
for numerical_feature in numerical_features:
    ax = plt.subplot(12, 3, num)
    sns.boxplot(df[numerical_feature])
    plt.xlabel(numerical_feature)
    num += 1
plt.show()

In [ ]:
def count_outliers_iqr(df, column):
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1

    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]

    return outliers[column].count()

In [ ]:
count_outliers_iqr(df, ['creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium'])

Dapat diketahui bahwa terdapat beberapa data yang outlier pada atribut creatinine_phosphokinase,ejection_fraction, platelets, serum_creatinine, dan serum_sodium

**Handling Data Outliers**

In [ ]:
outliers = ['creatinine_phosphokinase', 'ejection_fraction',
            'platelets', 'serum_creatinine', 'serum_sodium']

for i in outliers:
    q1 = df[i].quantile(0.25)
    q3 = df[i].quantile(0.75)

    iqr = q3 - q1
    mini = q1 - 1.5 * iqr
    maxi = q3 + 1.5*iqr

    df[i] = np.where(
        df[i] > maxi,
        maxi,
        np.where(
            df[i] < mini,
            mini,
            df[i]
        )
    )

In [ ]:
count_outliers_iqr(df, ['creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium'])

In [ ]:
# visualisasi numerical features
plt.figure(figsize=(20, 60), facecolor='white')
num = 1
for numerical_feature in numerical_features:
    ax = plt.subplot(12, 3, num)
    sns.boxplot(df[numerical_feature])
    plt.xlabel(numerical_feature)
    num += 1
plt.show()

## Analisis age vs death_event

In [ ]:
death_event_1 = df[df['death_event'] == 'Yes']

plt.figure(figsize=(10, 6))
sns.histplot(data=death_event_1, x='age', kde=True, bins=20)
plt.title('Distribusi Umur Pasien dengan Death Event == Yes')
plt.xlabel('Umur')
plt.ylabel('Jumlah Pasien')

plt.show()

Dapat diketahui bahwa pasien yang meninggal karena penyakit jantung paling banyak berumur 60-an tahun

## Analisis anaemia vs death_event

In [ ]:
df.groupby('death_event')['anaemia'].value_counts().plot(kind='barh')

Dapat diketahui bahwa lebih banyak pasien yang tidak meninggal dan tidak memiliki riwayat anemia

## Analisis diabetes vs death_event

In [ ]:
df.groupby('death_event')['diabetes'].value_counts().plot(kind='barh')

Dapat diketahui bahwa lebih banyak pasien yang tidak meninggal dan tidak memiliki riwayat diabetes

## Analisis high_blood_pressure vs death_event

In [ ]:
df.groupby('death_event')['high_blood_pressure'].value_counts().plot(kind='barh')

Dapat diketahui bahwa lebih banyak pasien yang tidak meninggal dan tidak memiliki riwayat tekanan darah tinggi

## Analisis smoking vs death_event

In [ ]:
df.groupby('death_event')['smoking'].value_counts().plot(kind='barh')

Dapat diketahui bahwa lebih banyak pasien yang tidak meninggal dan tidak merokok

## Analisis serum_cretinin dan serum_sodium vs death_event

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(12,5))

sns.boxplot(data=df, x='death_event', y='serum_creatinine', ax=axs[0])
axs[0].set_title('Distribusi Serum Creatinine Berdasarkan Kelas DEATH_EVENT')
axs[0].set_xlabel('Kelas DEATH_EVENT')
axs[0].set_ylabel('Serum Creatinine')

sns.boxplot(data=df, x='death_event', y='serum_sodium', ax=axs[1])
axs[1].set_title('Distribusi Serum Sodium Berdasarkan Kelas DEATH_EVENT')
axs[1].set_xlabel('Kelas DEATH_EVENT')
axs[1].set_ylabel('Serum Sodium')

plt.show()

Dapat diketahui bahwa :
1. Pasien yang meninggal biasanya memiliki kadar kreatinin lebih tinggi 
2. Pasien yang meninggal biasanya memiliki kadar sodium yang lebih rendah

# Data Pre-processing

## Encode

In [ ]:
# Ubah kedalam bentuk numerik
encoded_data = LabelEncoder()

for cat in categorical_features:
        if cat in df.columns.values:
            df[cat] = encoded_data.fit_transform(df[cat])

df.head()

## Korelasi Data

In [ ]:
# Cek korelasi
corr_mat = df.corr()
fig = plt.figure(figsize=(16, 8))
sns.heatmap(corr_mat, annot=True)

# Pendefinisian Model

## Train-Test-Split

Menentukan variabel bebas dan variabel terikat :

x = variabel bebas = age, anaemia, creatinine_phosphokinase, diabetes, ejection_fraction, high_blood_pressure, platelets, serum_creatinine, serum_sodium, sex, smoking dan time	
y = variabel terikat = death_event

In [ ]:
X = df.drop('death_event', axis=1)
y = df['death_event']

Membagi data menjadi training dan testing dengan training sebesar 70% dan testing sebesar 30%

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Pelatihan Model

## Handle Imbalanced Data

In [ ]:
plt.figure(figsize=(10, 4), facecolor="white")
plt.subplots_adjust(hspace=1)

ax = plt.subplot(2, 1, 1)
sns.countplot(y='death_event', data=df)
ax.bar_label(ax.containers[0])
plt.xlabel('death_event')
plt.title('death_event')

plt.show()

Dapat diketahui bahwa kedua grafik diatas masih tergolong tidak balance atau imbalanced karena perbedaannya terlalu jauh. Karena data tergolong imbalanced perlu dilakukan balancing data. Pada kasus ini menggunakan oversampling, yaitu menyeimbangkan dataset dengan menambah jumlah sampel pada kelas minoritas sehingga distribusi kelas menjadi lebih seimbang.

In [ ]:
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X, y)

In [ ]:
plt.figure(figsize=(10, 4), facecolor="white")
plt.subplots_adjust(hspace=1)

ax = plt.subplot(2, 1, 1)
sns.countplot(y=y_resampled, data=df)
ax.bar_label(ax.containers[0])
plt.xlabel('death_event')
plt.title('death_event')

plt.show()

In [ ]:
# melakukan split pada X_resampled dan y_resampled
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3)

In [ ]:
naiveBayes = GaussianNB()
knn = KNeighborsClassifier()
randomForest = RandomForestClassifier()
logReg = LogisticRegression()

## Ensemble Model

In [ ]:
# Inisialisasi Voting Classifier
voting = VotingClassifier(estimators=[
    ('naive_bayes', naiveBayes),
    ('knn', knn),
    ('random_forest', randomForest),
    ('logistic_regression', logReg)
])

# Melatih model dengan data training
voting.fit(X_train, y_train)

# Mengevaluasi model dengan data testing
score = voting.score(X_test, y_test)

# Evaluasi Model

In [ ]:
print('Akurasi model Voting Classifier:', score)

In [ ]:
voting_prediction = voting.predict(X_test)

cm_voting = confusion_matrix(y_test, voting_prediction)
fig,ax = plt.subplots(figsize=(4,4))
plt.title("Confusion Matrix Logistic Regression")

ax.imshow(cm_voting)
ax.grid(False)
ax.xaxis.set(ticks=(0,1),ticklabels = ('Prediksi 0s','Prediksi 1s'))
ax.yaxis.set(ticks=(0,1),ticklabels = ('Aktual 0s','Aktual 1s'))

for i in range(2):
    for j in range(2):
        ax.text(j,i,cm_voting[i,j],ha='center',va='center',color='red')

plt.show()

# Model Inference

In [ ]:
df.columns

In [ ]:
df_new = pd.DataFrame({
    'age': [61, 58, 47],
    'anaemia': [0, 1, 1],
    'creatinine_phosphokinase': [200, 400, 500],
    'diabetes': [0, 0, 1],
    'ejection_fraction': [32, 45, 39],
    'high_blood_pressure': [1, 0, 1],
    'platelets': [217000, 318000, 265000],
    'serum_creatinine': [0.9, 1, 1.2],
    'serum_sodium': [134, 137, 139],
    'sex': [1, 1, 0],
    'smoking': [1, 0, 0],
    'time': [2, 500, 150]
})

In [ ]:
df_new2 = pd.DataFrame({
    'age': [X_test.iloc[0,0], X_test.iloc[3,0], X_test.iloc[4,0]],
    'anaemia': [X_test.iloc[0,1], X_test.iloc[3,1], X_test.iloc[4,1]],
    'creatinine_phosphokinase': [X_test.iloc[0,2], X_test.iloc[3,2], X_test.iloc[4,2]],
    'diabetes': [X_test.iloc[0,3], X_test.iloc[3,3], X_test.iloc[4,3]],
    'ejection_fraction': [X_test.iloc[0,4], X_test.iloc[3,4], X_test.iloc[4,4]],
    'high_blood_pressure': [X_test.iloc[0,5], X_test.iloc[3,5], X_test.iloc[4,5]],
    'platelets': [X_test.iloc[0,6], X_test.iloc[3,6], X_test.iloc[4,6]],
    'serum_creatinine': [X_test.iloc[0,7], X_test.iloc[3,7], X_test.iloc[4,7]],
    'serum_sodium': [X_test.iloc[0,8], X_test.iloc[3,8], X_test.iloc[4,8]],
    'sex': [X_test.iloc[0,9], X_test.iloc[3,9], X_test.iloc[4,9]],
    'smoking': [X_test.iloc[0,10], X_test.iloc[3,10], X_test.iloc[4,10]],
    'time': [X_test.iloc[0,11], X_test.iloc[3,11], X_test.iloc[4,11]]
})

In [ ]:
df_new.head()

In [ ]:
df_new2.head()

In [ ]:
voting_prediction_new = voting.predict(df_new)
voting_prediction_new

In [ ]:
voting_prediction_new2 = voting.predict(df_new2)
voting_prediction_new2

# Pengambilan Kesimpulan

**Faktor-Faktor** 

Dapat diketahui faktor-faktor yang mempengaruhi keselamatan pasien dengan riwayat penyakit jantung adalah sebagai berikut :

1. age : umur pasien
2. anaemia : apakah ada pengurangan haemoglobin
3. creatinine_phosphokinase : level enzim CPK dalam mcg/L
4. diabetes : apakah pasien punya riwayat diabetes
5. ejection_fraction : persentase darah yang meninggalkan jantung dalam persentasi di setiap kontraksi jantung
6. high_blood_pressure : apakah pasien punya darah tinggi
7. platelets : jumlah platelet di darah dalam kiloplatelets/mL
8. serum_creatinine : level serum creatinine di darah dalam mg/dL
9. serum_sodium : level serum sodium di darah dalam mEq/L
10. sex : apakah pasien pria atau wanita
11. smoking : apakah pasien merokok
12. time : waktu dalam hari untuk follow-up
13. DEATH_EVENT : apakah pasien sudah meninggal saat waktu follow-up
    
**Akurasi** 

Berdasarkan pelatihan ensemble model yang telah dilakukan dapat diperoleh akurasi dari model Voting dari keempat model(Naive Bayes, KNN, Random Forest, dan Logistic Regression) adalah sebesar 87.70 %

**Wawasan**

Insight yang ditemukan dalam prediksi keselamatan pasien dengan riwayat penyakit jantung adalah sebagai berikut :
1. Dapat diketahui bahwa pasien yang meninggal karena penyakit jantung paling banyak berumur 60-an tahun
2. Dapat diketahui bahwa pasien yang memiliki penyakit jantung kebanyakan berjenis kelamin laki-laki
3. Dapat diketahui bahwa pasien yang meninggal biasanya memiliki kadar kreatinin lebih tinggi dan/atau kadar sodium yang lebih rendah 